# Antenna Calculations in SpaceLink

This notebook demonstrates the antenna capabilities available in the SpaceLink library.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

%matplotlib widget

import astropy.units as u
from spacelink.core import antenna
from spacelink.core.units import to_linear

First up we have the gain of a parabolic dish antenna.

In [2]:
frequency = 2.25 * u.GHz
dish_diameter = 3.7 * u.m
radiation_efficiency = 0.65 * u.dimensionless
gain = antenna.dish_gain(dish_diameter, frequency, radiation_efficiency)
print(f"The gain of a {dish_diameter:0.1f} dish at {frequency:0.2f} is {gain:0.1f}")

The gain of a 3.7 m dish at 2.25 GHz is 36.9 dB


Next we can calculate the loss resulting from axial ratio mismatch between two antennas
assuming worst-case orientations of the polarization ellipses (90° apart).

In [5]:
axial_ratio = 3.0 * u.dB
polarization_loss = antenna.polarization_loss(axial_ratio, axial_ratio)
print(
    f"Polarization loss for {axial_ratio:.2f} dB axial ratio: {polarization_loss:.2f}"
)

Polarization loss for 3.00 dB dB axial ratio: 0.51 dB


Below is a visualization of the polarization ellipse associated with an antenna 
radiation pattern in 
[spherical coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system), 
where $\theta$ is the polar angle and $\phi$ is the azimuthal angle. The polarization 
ellipse is shown in a plane tangent to the sphere known as the _local tangent plane_. 
The direction of propagation is out of the screen. 


Polarization is here parameterized by
* Tilt angle of the major axis with respect to $\hat{\theta}$,
* Axial ratio, which is the ratio of the major and minor axes of the ellipse, and
* Handedness, which determines the direction of the E-field rotation.

In [ ]:
fig_pol, ax_pol = plt.subplots(figsize=(6, 6))


@interact(
    tilt_angle=FloatSlider(
        value=0,
        min=0,
        max=360,
        step=1,
        description="Tilt Angle (deg)",
        style={"description_width": "150px"},
        layout={"width": "400px"},
    ),
    axial_ratio=FloatSlider(
        value=0,
        min=0,
        max=30,
        step=0.1,
        description="Axial Ratio (dB)",
        style={"description_width": "150px"},
        layout={"width": "400px"},
    ),
    rotation_sense=Dropdown(
        options=[
            ("Left", antenna.Handedness.LEFT),
            ("Right", antenna.Handedness.RIGHT),
        ],
        value=antenna.Handedness.RIGHT,
        description="Rotation Sense",
        style={"description_width": "150px"},
        layout={"width": "400px"},
    ),
)
def plot_polarization_ellipse(tilt_angle, axial_ratio, rotation_sense):

    polarization = antenna.Polarization(
        tilt_angle * u.deg, to_linear(axial_ratio * u.dB), rotation_sense
    )
    jones = polarization.jones_vector

    t = np.linspace(0, 2 * np.pi, 100)
    ellipse = np.real(jones[:, np.newaxis] * np.exp(1j * t))

    ax_pol.clear()
    ax_pol.plot(ellipse[1, :], ellipse[0, :], linewidth=2)
    ax_pol.set_xlim(-1.2, 1.2)
    ax_pol.set_ylim(-1.2, 1.2)
    ax_pol.set_aspect("equal")
    ax_pol.set_xlabel(r"$\hat{\phi}$")
    ax_pol.set_ylabel(r"$\hat{\theta}$")
    ax_pol.set_title("Polarization Ellipse")
    ax_pol.invert_yaxis()
    ax_pol.grid(True, alpha=0.3)

    # Add arrow to show rotation direction
    ax_pol.annotate(
        "",
        xy=(ellipse[1, -1], ellipse[0, -1]),
        xytext=(ellipse[1, -2], ellipse[0, -2]),
        arrowprops=dict(arrowstyle="-|>", color="tab:blue", lw=2),
    )